**Checking for GPU access in Colab and python version accessed.**

In [1]:
#===================================
#part 1 Gloved vs Ungloved Detection
#===================================
#Technical Assesssment for Bitz-Tech-Analytics
#Aryaman Jain
#-----------------------------------

#Setup
!nvidia-smi  #to check if gpu is available in colab
!python --version
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

# Your plotting code here
plt.savefig('output_image.png')

Wed Sep 24 14:11:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Installing dependencies and YOLOV5**

In [1]:
%cd /content
!rm -rf yolov5  # agar half-broken repo bacha hai toh hata de
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17577, done.
remote: Total 17577 (delta 0), reused 0 (delta 0), pack-reused 17577 (from 1)
Receiving objects: 100% (17577/17577), 16.82 MiB | 29.85 MiB/s, done.
Resolving deltas: 100% (11978/11978), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00


**Installing pre-existing dataset from Roboflow.**

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QNdZcOils29F04VExHeP")
project = rf.workspace("abbos-aliboev").project("gloves-detection-huhre")
version = project.version(12)
dataset = version.download("yolov5")
print("Dataset downloaded at:", dataset.location)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Gloves-Detection-12 in yolov5pytorch:: 100%|██████████| 7179/7179 [00:00<00:00, 7666.62it/s]

Dataset downloaded at: /content/yolov5/Gloves-Detection-12


**Training the Model**

In [3]:
# Train YOLOv5s (small model for speed)
!python train.py --img 640 --batch 16 --epochs 30 \
--data /content/yolov5/Gloves-Detection-12/data.yaml \
--weights yolov5s.pt --project ../runs/train --name glove_detector
# After training, best weights will be saved here:
# runs/train/glove_detector/weights/best.pt

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      16/29      4.47G    0.03393    0.01733   0.001892         53        640:  58% 105/180 [00:38<00:22,  3.29it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      16/29      4.47G    0.03396    0.01732   0.001881         50        640:  59% 106/180 [00:38<00:24,  3.02it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      16/29      4.47G    0.03391    0.01732   0.001868         58        640:  59% 107/180 [00:38<00:22,  3.30it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(

**Classifying the training and validation data for two classes-(glove,no-gloves)**

In [4]:
%%writefile /content/yolov5/Gloves-Detection-12/data.yaml
train: /content/yolov5/Gloves-Detection-12/train/images
val: /content/yolov5/Gloves-Detection-12/valid/images

nc: 2
names: ['glove', 'no-gloves']

Overwriting /content/yolov5/Gloves-Detection-12/data.yaml


 **!cat prints the contents in the file directly.**

In [42]:
!cat /content/yolov5/Gloves-Detection-12/data.yaml

train: /content/yolov5/Gloves-Detection-12/train/images
val: /content/yolov5/Gloves-Detection-12/valid/images

nc: 2
names: ['glove', 'no-gloves']


**Using !ls to display the contents in the current directory**

In [43]:
!ls /content/yolov5/Gloves-Detection-12

data.yaml  README.dataset.txt  README.roboflow.txt  train  valid


**Running the best.pt to detect glove in validation images and then saving the annotated images and label to output\annotated folder.**

In [46]:
# Create folders for outputs/logs
!python /content/yolov5/detect.py --weights /content/runs/train/glove_detector/weights/best.pt \
--source {dataset.location}/valid/images \
--conf-thres 0.35 \
--project /content/output \
--name annotated \
--exist-ok

detect: weights=['/content/runs/train/glove_detector/weights/best.pt'], source=/content/yolov5/Gloves-Detection-12/valid/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.35, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/output, name=annotated, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-432-g725b922e Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/716 /content/yolov5/Gloves-Detection-12/valid/images/-_PNG.rf.2d368c9966b2f6255e5212344e7605e7.jpg: 640x640 1 no-gloves, 12.2ms
image 2/716 /content/yolov5/Gloves-Detection-12/valid/images/00004_jpg.rf.db7c8a86902c3f8c4714134f5b5e2fdc.jpg: 6

  **Listing all files and folder presnt in my directory.**

In [47]:
import os
print(os.listdir(dataset.location))  # should show 'train', 'valid', etc.

['data.yaml', 'README.dataset.txt', 'README.roboflow.txt', 'train', 'valid']


**Storing detection results in json**

In [49]:
import torch, json
from pathlib import Path
import cv2

#load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/runs/train/glove_detector/weights/best.pt')

#directory with validation images
img_dir = Path(dataset.location) / "valid/images"

# create the logs directory if it doesn't exist
Path("logs").mkdir(exist_ok=True)

for img_path in img_dir.glob("*.jpg"):
  results = model(str(img_path))
  detections = []

  for *xyxy, conf, cls in results.xyxy[0].tolist():
    x1, y1, x2, y2 = map(int, xyxy)
    detections.append({
        "label": model.names[int(cls)],
        "confidence": round(float(conf), 2),
        "box": [x1, y1, x2, y2]
    })

  log_obj = {
      "filename": img_path.name,
      "detections": detections
  }

  with open(f"logs/{img_path.stem}.json", "w") as f:
    json.dump(log_obj, f, indent = 2)

print("Json logs saved in logs/")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-9-24 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', a

Json logs saved in logs/


`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


**Using shutil to copy my annotated images and some detection logs in submission folder.**

In [50]:
from pathlib import Path
import shutil

img_folder = Path("output/annotated")
log_folder = Path("logs")
submission_output = Path("../submission/Part_1_Glove_Detection/output")
submission_logs = Path("../submission/Part_1_Glove_Detection/logs")

submission_output.mkdir(parents=True, exist_ok=True)
submission_logs.mkdir(parents=True, exist_ok=True)

# Copy only images that have JSON logs
for img_path in img_folder.glob("*.jpg"):
    log_path = log_folder / f"{img_path.stem}.json"
    if log_path.exists():
        shutil.copy(img_path, submission_output)
        shutil.copy(log_path, submission_logs)

**Verifiying the output/annotated consists of images**

In [51]:
!ls -l output/annotated

total 61920
-rw-r--r-- 1 root root 144484 Sep 24 16:14 00004_jpg.rf.db7c8a86902c3f8c4714134f5b5e2fdc.jpg
-rw-r--r-- 1 root root 183936 Sep 24 16:14 00035_jpg.rf.40d9fd0108c6648ff5fd56ba57d7826e.jpg
-rw-r--r-- 1 root root 168149 Sep 24 16:14 00126_jpg.rf.58135d33597d2a787b7e05f5a476b054.jpg
-rw-r--r-- 1 root root 196403 Sep 24 16:14 00422_jpg.rf.0bf0041505a5d073fc1ecac9de6c8e7b.jpg
-rw-r--r-- 1 root root  88342 Sep 24 16:14 00486_jpg.rf.e8a343730a922b1be33ced242da6a1e7.jpg
-rw-r--r-- 1 root root  46884 Sep 24 16:14 0054789_callaway-tour-authentic-golf-glove-360x360_jpeg_jpg.rf.44eefde21ea4d8c6e25709884b75b676.jpg
-rw-r--r-- 1 root root 122830 Sep 24 16:14 105_JPG_jpg.rf.b51ef0f2eae44decb327f2aa7b32dee6.jpg
-rw-r--r-- 1 root root  87391 Sep 24 16:14 114_jpeg.rf.104ec420e95a0b6bbedb011b77979760.jpg
-rw-r--r-- 1 root root  62545 Sep 24 16:14 115_jpg.rf.b111d0f59c460b1d864655c46fd30409.jpg
-rw-r--r-- 1 root root  50340 Sep 24 16:14 121IMG__jpg.rf.9ef85d18243bdfdc88a0f2413c80b758.jpg
-rw-r--

**Cheking the output and log contents in my submission folder.**

In [52]:
# List files in the output folder
!ls -l ../submission/Part_1_Glove_Detection/output
# List files in the logs folder
!ls -l ../submission/Part_1_Glove_Detection/logs

total 61920
-rw-r--r-- 1 root root 144484 Sep 24 16:15 00004_jpg.rf.db7c8a86902c3f8c4714134f5b5e2fdc.jpg
-rw-r--r-- 1 root root 183936 Sep 24 16:15 00035_jpg.rf.40d9fd0108c6648ff5fd56ba57d7826e.jpg
-rw-r--r-- 1 root root 168149 Sep 24 16:15 00126_jpg.rf.58135d33597d2a787b7e05f5a476b054.jpg
-rw-r--r-- 1 root root 196403 Sep 24 16:15 00422_jpg.rf.0bf0041505a5d073fc1ecac9de6c8e7b.jpg
-rw-r--r-- 1 root root  88342 Sep 24 16:15 00486_jpg.rf.e8a343730a922b1be33ced242da6a1e7.jpg
-rw-r--r-- 1 root root  46884 Sep 24 16:15 0054789_callaway-tour-authentic-golf-glove-360x360_jpeg_jpg.rf.44eefde21ea4d8c6e25709884b75b676.jpg
-rw-r--r-- 1 root root 122830 Sep 24 16:15 105_JPG_jpg.rf.b51ef0f2eae44decb327f2aa7b32dee6.jpg
-rw-r--r-- 1 root root  87391 Sep 24 16:15 114_jpeg.rf.104ec420e95a0b6bbedb011b77979760.jpg
-rw-r--r-- 1 root root  62545 Sep 24 16:15 115_jpg.rf.b111d0f59c460b1d864655c46fd30409.jpg
-rw-r--r-- 1 root root  50340 Sep 24 16:15 121IMG__jpg.rf.9ef85d18243bdfdc88a0f2413c80b758.jpg
-rw-r--

**Display and then save images using matplotlib and openCV**

In [56]:
import cv2
import json
from matplotlib import pyplot as plt
from pathlib import Path

# Paths
img_folder = Path("../submission/Part_1_Glove_Detection/output")
log_folder = Path("../submission/Part_1_Glove_Detection/logs")

# Get all images that have a matching JSON
sample_images = []
for img_path in img_folder.glob("*.jpg"):
    log_path = log_folder / f"{img_path.stem}.json"
    if log_path.exists():
        sample_images.append(img_path)

# Take first 5 images
sample_images = sample_images[:5]

# Display images
for img_path in sample_images:
    # Read image
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Cannot read {img_path}")
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Read JSON log
    log_path = log_folder / f"{img_path.stem}.json"
    with open(log_path) as f:
        data = json.load(f)

    # Draw bounding boxes
    for det in data["detections"]:
        x1, y1, x2, y2 = det["box"]
        label = det["label"]
        conf = det["confidence"]
        color = (0, 255, 0) if label.lower() == "glove" else (255, 0, 0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, f"{label} {conf}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the figure to a file instead of displaying it
    output_filename = f"processed_{img_path.name}"
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title(img_path.name)
    plt.savefig(output_filename, bbox_inches='tight')
    plt.close() # Close the figure to free up memory

    print(f"Saved {output_filename}")

Saved processed_IMG_4864_57-59sec_jpg.rf.0180fd11de5295f4d94cf4f20aaca521.jpg
Saved processed_video_BC5_image_142_png_jpg.rf.bdd9d1d9862a6cb3c52d2cbad858898a.jpg
Saved processed_IMG_4697_36-40sec_jpg.rf.1467f817eadd6f1f558956651c147443.jpg
Saved processed_youtube-224_jpg.rf.58a573b67554427506f077eb0ec9a619.jpg
Saved processed_IMG_5718_24-32sec_jpg.rf.0082f730444ed6c890661e28be0f2ca4.jpg


**To check which files or image actaullly existis as json**

In [54]:
import os
print(sorted(os.listdir("../submission/Part_1_Glove_Detection/logs")))

['-_PNG.rf.2d368c9966b2f6255e5212344e7605e7.json', '00004_jpg.rf.db7c8a86902c3f8c4714134f5b5e2fdc.json', '00035_jpg.rf.40d9fd0108c6648ff5fd56ba57d7826e.json', '00126_jpg.rf.58135d33597d2a787b7e05f5a476b054.json', '00422_jpg.rf.0bf0041505a5d073fc1ecac9de6c8e7b.json', '00486_jpg.rf.e8a343730a922b1be33ced242da6a1e7.json', '0054789_callaway-tour-authentic-golf-glove-360x360_jpeg_jpg.rf.44eefde21ea4d8c6e25709884b75b676.json', '105_JPG_jpg.rf.b51ef0f2eae44decb327f2aa7b32dee6.json', '114_jpeg.rf.104ec420e95a0b6bbedb011b77979760.json', '115_jpg.rf.b111d0f59c460b1d864655c46fd30409.json', '12-98sec_IMG_5568_jpg.rf.eaae92982c591d02573f4ba5d20bd29a.json', '121IMG__jpg.rf.9ef85d18243bdfdc88a0f2413c80b758.json', '126-125sec_IMG_5560_jpg.rf.d83a9cb71537d8fbba2f675d4274a7d7.json', '12_JPG.rf.3160df163a6a516b16f007dfe1bdc98e.json', '12_jpeg.rf.d9a7b82cc1a808d821eb40981572c2d1.json', '131_jpg.rf.10b9061432b7f292082d5ef07e645f86.json', '132_jpg.rf.0cc9ccde8b538e1dd38f3a87144c79e9.json', '142-1422613_wint